Importing Necessary Packages

In [2]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix

# Planetary Computer Tools
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
pc.settings.set_subscription_key('xx6tg-6yxmd-gch4d-xf7y7-gk6fz')

# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()

In [3]:
# This is where the training labels are stored

crop_presence_data = pd.read_csv("crop_loc_data.csv")
crop_presence_data.head()


,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


In [10]:
def get_sentinel_data(latlong,time_slice,assets):
    '''
    Returns VV and VH values for a given latitude and longitude 
    Attributes:
    latlong - A tuple with 2 elements - latitude and longitude
    time_slice - Timeframe for which the VV and VH values have to be extracted
    assets - A list of bands to be extracted
    '''

    latlong=latlong.replace('(','').replace(')','').replace(' ','').split(',')

    # Creating bounding box
    box_size_deg = 0.10
    min_lon = float(latlong[1]) - box_size_deg/2
    min_lat = float(latlong[0]) - box_size_deg/2
    max_lon = float(latlong[1]) + box_size_deg/2
    max_lat = float(latlong[0]) + box_size_deg/2
    bbox_of_interest = (min_lon, min_lat, max_lon, max_lat)
    # bbox_of_interest = (float(latlong[1]) - .0000000000001, float(latlong[0]) - .0000000000001, float(latlong[1]) + .0000000000001, float(latlong[0]) + .0000000000001)
    print(bbox_of_interest)
    time_of_interest = time_slice

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
    )
    search = catalog.search(
        collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_interest
    )
    items = list(search.get_all_items())
    data = stac_load([items[0]], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
    vh = data["vh"].astype("float").values.tolist()[0][0]
    vv = data["vv"].astype("float").values.tolist()[0][0]
    return vh,vv

In [ ]:
## Function call to extract VV,VH Values
time_slice = "2020-03-20/2020-03-21"
assests = ['vh','vv', 'nir']
vh_vv = []
for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    print(coordinates)
    vh_vv.append(get_sentinel_data(coordinates,time_slice,assests))
vh_vv_data = pd.DataFrame(vh_vv,columns =['vh','vv', 'nir'])

In [ ]:
crop_data = pd.concat([crop_presence_data,vh_vv_data], axis=1)
crop_data = crop_data[['vh', 'vv', 'nir', 'Class of Land']]
crop_data.head()

In [ ]:
X = crop_data.drop(columns=['Class of Land']).values
y = crop_data ['Class of Land'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y,random_state=40)
X_train[0]